Delete all variables in the current environment (if you have already run some cells) - clean state

In [1]:
%reset

Import all necessary packages

In [2]:
import pandas as pd
import os
import networkx as nx
from matplotlib import pyplot as plt
import shutil

Replace with the path to the root folder of the project

In [3]:
rootdir_path = '/home/andreistoica12/research-internship'

Replace with the path to the folder where we store the dataset

In [4]:
data_path = '/home/andreistoica12/research-internship/data/covaxxy-csv'

Create 2 subfolders to store important files and graphs, respectively. If they already existed (from previous runnings of the project), delete the folders and their contents and create empty folders to store the current files and graphs, relevant to the current state of the project.

In [5]:
files_path = os.path.join(rootdir_path, 'files')
if os.path.exists(files_path):
   shutil.rmtree(files_path, ignore_errors=False, onerror=None)
os.makedirs(files_path)

graphs_path = os.path.join(rootdir_path, 'graphs')
if os.path.exists(graphs_path):
   shutil.rmtree(graphs_path, ignore_errors=False, onerror=None)
os.makedirs(graphs_path)

In [6]:
os.listdir(data_path)

['tweet_ids--2021-03-02.csv',
 'tweet_ids--2021-03-03.csv',
 'tweet_ids--2021-03-05.csv',
 'tweet_ids--2021-03-04.csv',
 'tweet_ids--2021-03-01.csv']